# Поиск, подготовка и анализ данных

### Автор: Ракитин Виталий, BD-21

In [77]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from tqdm import tqdm
import json

import requests
import re
from HTMLParser import HTMLParser
from lxml import html

## 1. Откачиваем данные с сайта otvet.mail.ru

Несложно заметить, что на сайте otvet.mail.ru все вопросы пронумерованы (по id), соответсвтенно и откачивать данные межно простым обходом по номерам. Ограничимся первыми 10млн. вопросами.

In [2]:
# headaers for http request
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Encoding' : 'gzip,deflate,sdch',
    'Accept-Language' : 'en-US,en;q=0.8',
}

html = HTMLParser()
PARSE_FILE = "data.csv"
READY_STR = "READY: {0} : {1} of {2} --- {3:.2f}%;"

def get(num, amount = 10):
    question = None
    answers = None
    try:
        request = requests.get('https://otvet.mail.ru/question/' + str(num), headers = HEADERS) 
        question = re.findall(u'<title>Ответы@Mail.Ru: (.*?)</title>', request.text)[0]
        question = html.unescape(question)
        question = question.encode('utf-8')
        
        answers = re.findall(u'<div class="a--atext atext" itemprop="text">(.*?)</div>', request.text)
        answers = map(lambda x: re.sub(u'<br><br>','\n', x), answers)
        answers = map(lambda x: re.sub(u'<.*>','\n', x), answers)
        answers = map(lambda x: re.sub(u'\n',' ', x), answers)
        answers = map(lambda x: html.unescape(x), answers)
        answers = map(lambda x: x.encode('utf-8'), answers)
        answers = [ans for ans in answers if len(ans) > 1]
        answers = answers[0:4]

    except:
        pass #Page does not exist
    return question, answers

def save(question, answers, question_id, file = PARSE_FILE):
    ''' save qusetion in file '''
    if question != None and answers != None:
        with open(file, "a") as file:
            
            file.write(str(question_id))
            file.write("\t\t")
            file.write(question)
            for answer in answers:
                file.write("\t\t")
                file.write(answer)
           
            file.write("\n")

def download(first, last, file = PARSE_FILE, frequency = 1000):
    good = 0
    bad = 0
    pages = xrange(first, last)
    for page_num in pages:
        question, answers = get(page_num)
        if question != None: 
            save(question, answers, page_num, file)
            good += 1
        else:
            bad += 1

        if (bad + good) % frequency == 0:
            print READY_STR.format(good, bad, last - first, 
                                   float(good+bad)/float(last-first)*100)

In [3]:
downloaded = True
if downloaded:
    itterator = pd.read_csv("parse.csv",sep = "\t\t", header=None, chunksize=10000)
    for table in itterator:
        break
else:
    download(0, 10000000, "parse.csv", 10000)

/Users/Vitaliy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [4]:
table.head()

,0,1,2,3,4,5
0,2,Какие документы нужны для получения загранпасп...,Список необходимых документов для получения за...,# Оригинал гражданского паспоpта для подачи до...,два документа достоинством 100 долларов.,Все
1,3,Где в Подмосковье или в Москве можно показатьс...,Можно в Солярисе:,Привет из 2017 года,Можно в Солярисе.,None
2,4,Как называется столица Албании?,Тирана.,Тирана!,Тирана,Тирана.
3,5,Хочу пострелять в тире. Где в Москве это можно...,Попробуй здесь:,Я тоже хочу! Возьми меня с собой!,на Октябрьском поле есть какой-то хороший тир ...,На Октябрьском
4,6,"Посоветуйте молодых и недорогих, но прикольных...","Что еще есть? А по мне, все равно выгоднее дв...",Слава Зверев :) Ну оочень молодой,Евгения Островская - P.S. это не моя родствен...,Зайцев =))


## 2. Анализ откаченных данных

1. Попробуем посмореть, что из себя представляют откаченные данные на небольшой выборке.

2. Построим словарь из используемых слов.

3. Токенизируем нашу выборку с помощью метода one-hot-encoding.

In [43]:
class one_hot_tokenizer():
    def __init__(self, is_stem = False):
        ''' 
            * self.vocabulary : {word : id}
            * self.back_vocabulary : {id : words}
            * self.words_count_dict : {word : word_counter}
            * self.words_counter - all words counter (int)
        Input: 
            * is_stem : to do stemming or not (False default)
        '''
        self.vocabulary = {}
        self.back_vocabulary = {}
        self.words_count_dict = {}
        
        stop_words = stopwords.words(u'russian')
        self.stem = SnowballStemmer("russian")
        self.stop_words = [self.stem.stem(i) for i in stop_words]
        self.words_counter = 0
        self.is_stem = is_stem
        
    def tokenize_me(self, text):
        ''' 
        Токенизируем предложение 
            * очищаем от мусора
            * удаляем стоп-слова
            * оставляем только русские буквы (слова) 
        Input:
            * text : to clean
        Output: 
            * tokens
        '''
        text = text.lower()
        tokens = re.findall(u'[а-яА-Я][а-яА-Я]*', text)

        #stemming
        if self.is_stem:
            tokens = [self.stem.stem(i) for i in tokens]


        #deleting stop_words
        new_tokens = [i for i in tokens if i not in self.stop_words]
        if len(new_tokens) < 2:
            new_tokens = tokens

        return new_tokens


    def create_word(self, word, build_voc = True):
        ''' 
            1. Дополняем словарь;
            2. Узнаём индекс слова;
        Input:
            * word 
            * build_voc - to add word in self.vocabulary
        Output:
            * word_id
        '''

        if word in self.vocabulary:
            word_id = self.vocabulary[word]
            if build_voc:
                self.words_count_dict[word_id] += 1
                self.words_counter += 1
        elif build_voc:
            word_id = len(self.vocabulary)
            self.vocabulary[word] = word_id
            self.back_vocabulary[word_id] = word
            self.words_count_dict[word_id] = 1
            self.words_counter += 1 
        else: 
            word_id = None
        return word_id
    def create_sentance(self, sentance, build_voc = True):
        ''' 
            Токенизируем и индексируем предложение 
        Input:
            * sentance
            * build_voc - to add words in self.vocabulary
        Output:
            * tokens of the sentance (list)
        '''
        if sentance is not None and len(sentance) > 0:
            try:
                sentance = sentance.decode('utf-8')
            except:
                pass
            sentance = self.tokenize_me(sentance)
            sentance = [self.create_word(word, build_voc) for word in sentance]
            return sentance
        else: 
            return None

    def fit(self, table, build_voc = True):
        ''' 
            Токенизируем данные и строим словарь
        Input:
            * table : data (pandas)
            * build_voc : to add words in self.vocabulary
        Output:
            * questions : tokenized questions (dict) 
            * answers : tokenized answers (dict) 
            * bad_sent : bad sentances (list) 
        '''

        questions = {}
        answers = {}
        bad_sent = []

        for sid, sent_id in tqdm(enumerate(table[0])):
            question = self.create_sentance(table[1][sid], build_voc)
            answer = self.create_sentance(table[2][sid], build_voc)

            if question != None and answer != None and len(question) > 0 and len(answer) > 0:
                questions[sent_id] = question
                answers[sent_id] = answer
            else:
                bad_sent += [sid]

            if build_voc:
                _ = [self.create_sentance(table[i][sid], build_voc) for i in [3,4,5]]
                
        return questions, answers, bad_sent
    
    def clean(self, sentence, frequency):
        ''' 
        Удаляем все редкие слова из токенизированного предложения
        Input:
            * sentence
            * frequency of a word
        Output:
            * new sentence
        '''
        new_sentence = [word for word in sentence if self.words_count_dict[word] > frequency]
        return new_sentence

    def clean_all(self, questions, answers, frequency = 5):
        ''' 
        Удаляем редкие слова из всех токенизированных пар вопрос/ответ 
        Input:
            * questions
            * answers
            * frequency of a word
        Output:
            * amount of lost sentances
            * new questions
            * new answers
        '''
        lost_count = 0
        new_q = {}
        new_a = {}
        words_counter = 0
        for it, sent in questions.items():
            q = self.clean(sent, frequency)
            a = self.clean(answers[it], frequency)
            if len(q) > 0 and len(a) > 0:
                new_q[it] = q
                new_a[it] = a
                words_counter += len(q) + len(a)
            else:
                lost_count += 1
        return new_q, new_a, lost_count, words_counter 
    
    def tokenize(self, sentance, frequency = 5):
        ''' 
        Токенизируем новое предложение
        Input:
            * sentance to tokenize
            * frequency of words
        Output:
            * tokens(list)
        '''
        sent = self.create_sentance(sentance)
        return self.clean(sent, frequency)

In [44]:
one_hot = one_hot_tokenizer()
questions, answers, bad_sent = one_hot.fit(table)

10000it [00:06, 1565.06it/s]


###### Посмотрим какие предложения потеряли.

In [45]:
# посмотрим сколько и чего потеряли
def loss_stat(bad_sent, amount = 5):
    '''
    Input:
        * bad_sent
        * amount : of results        
    '''
    
    print "Плохих сообщений :", len(bad_sent)
    print "Примеры: "
    if len(bad_sent) > amount:
        sentances = np.random.choice(np.array(bad_sent), amount)
    else:
        sentances = bad_sent
    
    for i in sentances:
        print "q = ", table[1][i]
        print "a = ", table[2][i]
        print "__________________"
        
loss_stat(bad_sent)

Плохих сообщений : 717
Примеры: 
q =  problema s tormozami
a =  Поменяйте колодки, и залейте жидкость, или прокачайте тормоза, и всё.
__________________
q =  EH BU DUNJA  SEN NIJE BELESEN?
a =  Фамилия наша, а гонит какой-то бред...
__________________
q =  Chelovek! Kto ty? Otkuda prishel? Kuda idesh'?
a =  Я пршел с почты@mail.ru и иду на overclockers.ru
__________________
q =  Quosque tandem abutere, Catilina, patientia nostra?
a =  Quam diu etiam furor iste nos eludet, quem ad finem sese effrenata iactabit audacia?  Cicerón, In Catilinam, I, I, 1
__________________
q =  pochemu muzhchiny zhenyatsya ne po lyubvi?
a =  Потому, что у женщин часто бывает прошлое - а каждый мужчина хочет быть первым у женщины.  А от Вас ,наверное, ушел молодой человек... И женился на другой? Сочувствую...
__________________


Видим, что дейтсительно встречается мусор:  транслит, пустые ответы, смайлы, ссылки.

##### Посмотрим полученный словарь, удалим из него редкие слова.

In [46]:
# посмотрим словарь без редких слов
def TOP(tokenizer, frequency = 5, amount = 10):
    print "Всего использовано слов: ", tokenizer.words_counter
    print "Размер словаря: ", len(tokenizer.vocabulary)
    
    voc = [(tokenizer.back_vocabulary[word_id], tokenizer.words_count_dict[word_id]) 
           for word_id in tokenizer.back_vocabulary
           if tokenizer.words_count_dict[word_id] > frequency]
    
    voc_top = sorted(voc, key = lambda x: x[1], reverse=True)[:amount]
    print "Без редких слов (реже {0}): {1}\n".format(frequency, len(voc))
    print "TOP {0}:".format(amount)
    for i in voc_top:
        print i[0],i[1]
    
TOP(one_hot)

Всего использовано слов:  364365
Размер словаря:  65996
Без редких слов (реже 5): 8175

TOP 10:
это 5607
если 4584
есть 2774
можно 2698
или 2663
только 1939
просто 1664
надо 1640
очень 1607
тебе 1594


Заметно:
1. огромное количество редких слов;
2. большую часть топа составляют stop-слова. 

Увы, stop-words из NTLK - не идеален. 

Посмотрим, что останется, если удалить редкие слова из реальных данных.

In [47]:
questions, answers, lost, words_count = one_hot.clean_all(questions, answers)

In [48]:
def full_stat(tokenizer, lost, words_count):
    print "Всего было: {0} слов;".format(tokenizer.words_counter)    
    print "Осталось: {0} слов".format(words_count)
    print "Всего потеряно: {0} слов".format(tokenizer.words_counter - words_count)
    print "Всего потеряно: {0} предложений".format(lost)

full_stat(one_hot, lost, words_count)

Всего было: 364365 слов;
Осталось: 126964 слов
Всего потеряно: 237401 слов
Всего потеряно: 350 предложений


К сожалению, у нас встречаются предложения, состоящие исключительно из стоп слов.

Так же большинство слов теряется.

Посмотрим, как работает наш токенизатор.

In [49]:
sentance = u"Привет! Как пройти до магазина?"
print one_hot.tokenize(sentance)

[132, 6800, 12169]


##### Остаётся проблема: размерность словаря (= размерность one-hot вектора)

Попробуем использовать стемминг для чистки слов.

In [28]:
one_hot_stem = one_hot_tokenizer(is_stem=True)
questions_stem, answers_stem, bad_sent_stem = one_hot_stem.fit(table)

10000it [02:04, 80.29it/s]


In [29]:
loss_stat(bad_sent_stem)

Плохих сообщений : 717
Примеры: 
q =  chto mne delat ja lublu svoego jenatogo nachaljnika
a =  А он тебя любит?? ?  
__________________
q =  est novost! poprosil cho bi ya dala miach. Chto delat dalshe?
a =  Расскажи стишок:  Это был соседский мяч!
__________________
q =  Где найти красивую девушку со знанием английского? 
a =  Ya podoidu? Pravda ya seichas v Amerike!
__________________
q =  on neznaet menia ia bous! Pomogiiiiiiiiiiiiite mne on samii luchshii v mire moi kotik!!!!!!!!!!!!!
a =  Я тебе уже писала, подойди и познакомся или напиши записку или позвони или через подруг
__________________
q =  где страничка с анализом динамики валют, которая была всегда доступна раньше по нажатию ссылки "катировки"?
a =  None
__________________


In [30]:
TOP(one_hot_stem)

Всего использовано слов:  295038
Размер словаря:  32133
Без редких слов (реже 5): 6327

TOP 10:
прост 1940
очен 1609
люб 1457
котор 1432
вопрос 1358
нужн 1280
человек 1234
дела 1206
жизн 1109
поч 1091


##### Заметно, что топ-слова стали более смысловыми.

In [31]:
questions_stem, answers_stem, lost_stem, words_count_stem = one_hot_stem.clean_all(questions_stem, answers_stem)

In [32]:
full_stat(one_hot_stem, lost_stem, words_count_stem)

Всего было: 295038 слов;
Осталось: 119699 слов
Всего потеряно: 175339 слов
Всего потеряно: 193 предложений


##### Размер словаря удалось сократить, но не значительно.

Так же из-за стемминга теряется смысл некоторых слов, что портит восприимчивость модели.

## 3. Векторизация слов с помощью word2vec

Попробуем токенизировать данные с помощью модели word2vec из библиотеки gensim.

Так же, для нормализации слов используем библиотеку Mystem.

In [50]:
from pymystem3 import Mystem
from gensim.models import KeyedVectors as Word2Vec
from collections import defaultdic

Далее будем использовать word2vec, обученный на веб-корпусе.

(источник: http://rusvectores.org/ru/models/#ruwikiruscorpora)

In [52]:
w2v = Word2Vec.load_word2vec_format("web.bin", binary=True)
normalizer = Mystem()

Посмотрим, как работает нормализатор 

In [56]:
ww = normalizer.analyze("красную")[0][u'analysis'][0]
for i in ww:
    print ww[i] 

красный
A=вин,ед,полн,жен


###### Выделим члены предложения, использованные в нашей модели word2vec

In [59]:
dd = defaultdict(lambda:0)
for k in tqdm(w2v.vocab):
    w = k.split("_")
    dd[w[1]] += 1
sorted(dd.items(), key = lambda x: x[1], reverse= True)

100%|██████████| 353608/353608 [00:00<00:00, 399711.48it/s]


[(u'S', 155983),
 (u'UNKN', 132466),
 (u'A', 35391),
 (u'V', 21982),
 (u'ADV', 6925),
 (u'INTJ', 161),
 (u'COM', 152),
 (u'PR', 110),
 (u'ANUM', 87),
 (u'PART', 85),
 (u'ADVPRO', 72),
 (u'CONJ', 62),
 (u'NUM', 56),
 (u'APRO', 42),
 (u'SPRO', 34)]

Теперь мы можем легко приводить слова к виду word2vec, нормализуя слова и определяя члены предложения с помощью Mystem.

In [66]:
def tow2v(word):
    ''' приведём слово к виду word2vec '''
    try:
        word = word.encode('utf-8')
    except:
        pass
    res = []
    for x in normalizer.analyze(word):
        if ('analysis' in x) and len(x['analysis']) > 0:
            txt = x['analysis'][0]['lex']
            tag = x['analysis'][0]['gr'].split("=")[0].split(",")[0]
            res.append(u"{0}_{1}".format(txt, tag))
    return res

def vectorize(text):
    ''' векторизуем предложение '''
    text = tow2v(text)
    if len(text) > 0:
        for i in text:
            print u"Слово {0} есть в w2v: {1}".format(i,i in w2v.vocab)
        return [w2v.word_vec(word) for word in text if word in w2v.vocab]
    return None

In [67]:
def normalize_word(word):
    if len(word.split(' ')) > 1 or len(word) < 2:
        return word

    norm = normalizer.analyze(word)[0]
    if 'analysis' in norm and len(norm['analysis']) > 0:
        return norm['analysis'][0]['lex']  
    else:
        if 'text' in norm:
            return norm['text']

In [70]:
print "Длина полученного предложения: {0}".format(len(vectorize('полёт кукушки в чебурашкой')))

Слово полет_S есть в w2v: True
Слово кукушка_S есть в w2v: True
Слово в_PR есть в w2v: False
Слово чебурашка_S есть в w2v: True
Длина полученного предложения: 3


In [73]:
print u"Длина вектора в нашем случае всегда будет фиксирована: ", w2v.vector_size

Длина вектора в нашем случае всегда будет фиксирована:  500


##### Результаты уже не могут не радовать! 

## 4. Поиск синонимов

Однако: задавая вопросы, люди часто используют не одни и те же слова, а заменяют их на синонимы. Поэтому, попробуем сделать поиск синонимов.

В интернете удалось обнаружить несколько неплохих словарей: 
1. Контекстный, обученный на интернет ресурсах (имеются синонимы для фраз)
2. Литературный.

In [76]:
# построим словарь синонимов
class synonims():
    ''' Построение словаря синонимов '''
    
    def __init__(self, w2v_file = "web.bin"):
        '''
        Input:
            * w2v_file : word2vec model file
        '''
        self.rFilter = re.compile(u"""[\'\.\,\!\"\№\;\%\:\?\@\$\^\*\&\(\)\_\+\~]""")
        self.spaceFilter = re.compile(u'\s+')
        self.w2v = Word2Vec.load_word2vec_format(w2v_file, binary=True)
        self.normalizer = Mystem()
        self.synonims_dict = None
        
    def load(self, filename = "synonims.json"):
        '''
        Загружаем словарь из файла
        Input:
            * filename
        '''
        with open(filename, 'r') as fp:
            self.synonims_dict = json.load(fp)
    
    def save(self, filename = "synonims.json"):
        '''
        Сохраняем словарь в файл
        Input:
            * filename
        '''
        with open(filename, 'w') as fp:
            json.dump(self.synonims_dict, fp)
        
    def create(self, 
               filename = "synmaster.txt", 
               code = 'CP1251', 
               spliter = "|",
               subspliter = None,
               max_syn_len = 20,
               save = True,
               save_filename ="synonims.json"
              ):
        '''
        Создаём/дополняем словарь используя готовый файл
        слово синоним1 синоним2 ... синонимN
        Input:
            * filename
            * code of the file
            * spliter : word | synonims
            * subspliter : spliter between synonims
            * max_syn_len : maximum amount of synonims
            * save : to save results
            * save_filename : where to save results
        '''
        
        with open(filename, "r") as f:
            text = f.read()
            text = text.decode(code)

        text = text.split("\n")
        synonims = defaultdict(lambda: list())
        
        for line in tqdm(text):
            syns = line.split(spliter)
            if subspliter is not None:
                try:
                    syns = [syns[0]] + syns[1].split(subspliter)
                except:
                    continue
            key = self.clean(syns[0])
            key = self.normalize_word(key)
            is_good_key = False

            if len(key) > 2:
                if len(key.split(" ")) < 2:
                    vect_key = self.tow2v(key) 
                    if len(vect_key) > 0:
                        vect_key = vect_key[0]
                        is_good_key = True

                for val in syns[1:]:
                    val = self.clean(val)
                    if len(val) >= 2:

                        if len(val.split(" ")) < 2:
                            val = self.normalize_word(val)
                            vect_val = self.tow2v(val) 

                            if len(vect_val) != 0 and (vect_val[0]  in self.w2v.vocab):    
                                vect_val = vect_val[0]
                                if vect_val not in synonims[key]:
                                    synonims[key].append(vect_val)

                        if is_good_key and vect_key not in synonims[val]:
                            synonims[val].append(vect_key) 
        
        print u"Всего получено {0} слов из {1}".format(len(synonims), filename)
        
        if self.synonims_dict is not None:
            self.join(synonims)
        else:
            self.synonims_dict = dict(synonims)
        
        if save:
            self.save(save_filename)
    
    def join(self, new_dict):
        '''
        Объединяем уже имеющийся словарь с новым
        Input:
            * new_dict
        '''
        for w in new_dict:
            if w not in self.synonims_dict:
                self.synonims_dict[w] = new_dict[w]
            else:
                for val in new_dict[w]:
                    if val not in self.synonims_dict[w]:
                        self.synonims_dict[w].append(val)
    
    def clean_base(self, max_syn_len = 20):
        ''' 
        Обрезаем количество синонимов для каждого слова 
        Input:
            * max_syn_len : maximum amount of synonims
        '''
        synonims = {}
        for word in tqdm(self.synonims_dict):
            if len(word.split(" ")) < 2:
                tok_vec = self.tow2v(word)
                if len(tok_vec) > 0 and tok_vec[0] in self.w2v.vocab:
                    syns = [(s, self.w2v.similarity(s, tok_vec[0])) for s in self.synonims_dict[word] if s in self.w2v.vocab]
                    sort_list = sorted(syns, key = lambda x: x[1], reverse=True)
                    synonims[word] = [w for w, _ in sort_list]
                else:
                    synonims[word] = self.synonims_dict[word]
            else:
                synonims[word] = self.synonims_dict[word]
                
            synonims[word] = synonims[word][:max_syn_len]
            #print len(self.synonims_dict[word]), len(synonims[word])

        print u"Всего было: {0} слов".format(len(self.synonims_dict))
        self.synonims_dict = synonims
        print u"Стало: {0} слов".format(len(self.synonims_dict))

        
    def clean(self, text):
        ''' 
        Очищаем пришедший текст 
        Input:
            * text
        '''
        text = text.lower()
        text = self.rFilter.sub('',text)
        text = self.spaceFilter.sub(' ', text)
        return text.strip()
    
    def tow2v(self, word):
        ''' 
        Приведём слово к виду word2vec 
        Input:
            * word
        '''
        try:
            word = word.encode('utf-8')
        except:
            pass
        
        res = []
        for x in self.normalizer.analyze(word):
            if ('analysis' in x) and len(x['analysis']) > 0:
                txt = x['analysis'][0]['lex']
                tag = x['analysis'][0]['gr'].split("=")[0].split(",")[0]
                res.append(u"{0}_{1}".format(txt, tag))
        return res

    def normalize_word(self, word):
        '''
        Нормализуем слово
        Input:
            * word
        
        '''
        if len(word.split(' ')) > 1 or len(word) < 2:
            return word

        norm = self.normalizer.analyze(word)[0]
        if 'analysis' in norm and len(norm['analysis']) > 0:
            return norm['analysis'][0]['lex']  
        else:
            if 'text' in norm:
                return norm['text']

    def find_synonim(self, text):
        '''
        Поиск синонимов для всех слов в тексте
        Input:
            * text
        Output:
            * dict of sinonims
        '''
        syns = {}
        tokens = self.create_synonim_tokens(text)
        for tok in tokens:
            if tok in self.synonims_dict:
                syns[tok] =  self.synonims_dict[tok]
        return syns
    
    def create_synonim_tokens(self, text):
        '''
        Разбиваем текст на слова и биграмы 
        Input:
            * text
        Output:
            * list of words and bigrams
        '''
        try:
            text = text.decode('utf-8')
        except:
            pass
        text = self.clean(text)
        text = text.split(' ')
        tokens = [self.normalize_word(word) for word in text]
        bis = []
        for word_ind in xrange(len(text) - 1):
            bis.append(u" ".join([text[word_ind], text[word_ind + 1]]))
        return tokens + bis
    
    def print_synonims(self, word):
        '''
        Вывести список синонимов для слова
        Input:
            * word
        '''
        
        syns = self.find_synonim(word)
        for w in syns:
            print "________\nSynonims for: ", w, " :"
            for j in syns[w]:
                print " > ", j

Посмотрим, что получилось:

In [79]:
syns = synonims()
syns.load("1.json")

In [81]:
syns.print_synonims("Праздник")

________
Synonims for:  праздник  :
 >  торжество_S
 >  празднество_S
 >  празднование_S
 >  масленица_S
 >  пасха_S
 >  юбилей_S
 >  рождество_S
 >  праздничек_S
 >  именины_S
 >  гуляние_S
 >  крестины_S
 >  карнавал_S
 >  сабантуй_S
 >  курбан-байрам_S
 >  благовещение_S
 >  крещение_S
 >  сретение_S
 >  комоедица_S
 >  веселие_S
 >  вечер_S


In [86]:
syns.print_synonims("разбиваем текст <> слова и @!    биграмы    обход")

________
Synonims for:  разбивать  :
 >  сломать_V
 >  разламывать_V
 >  разнести_V
 >  поломать_V
 >  расколачивать_V
 >  проламывать_V
 >  раздроблять_V
 >  разделять_V
 >  ломать_V
 >  поделить_V
 >  разгромить_V
 >  раскалывать_V
 >  расшибать_V
 >  сокрушать_V
 >  крушить_V
 >  разрушать_V
 >  раздолбать_V
 >  размозжить_V
 >  битый_A
 >  повреждать_V
________
Synonims for:  слово  :
 >  фраза_S
 >  словечко_S
 >  лексема_S
 >  термин_S
 >  выражение_S
 >  этимон_S
 >  словоформа_S
 >  антоним_S
 >  омоним_S
 >  говорить_V
 >  словцо_S
 >  изречение_S
 >  существительное_S
 >  речение_S
 >  речь_S
 >  омограф_S
 >  прилагательное_S
 >  обещание_S
 >  глагол_S
 >  пароним_S
________
Synonims for:  обход  :
 >  объезд_S
 >  патрулирование_S
 >  осмотр_S
 >  отвод_S
 >  огибание_S
 >  дозор_S
 >  осторожный_A
________
Synonims for:  текст  :
 >  шрифт_S
 >  текстовка_S
 >  слово_S
 >  стенограмма_S
 >  машинопись_S
 >  формулировка_S
 >  надпись_S
 >  аудиотекст_S
 >  словом_ADV
 >  

Видно, что результаты получаются очень неплохими. Можно приступать к подготовке к обучению модели.